The power of OpenAI function calling 🚀

In [ ]:
import os
import json
import openai
import requests
from ast import literal_eval
from IPython.display import JSON

In [ ]:
# Set environment variables
os.environ['OPENAI_API_KEY'] = '...'
os.environ['WEATHER_API_KEY'] = '...'

In [ ]:
open_ai_url = "https://api.openai.com/v1/chat/completions"
model = "gpt-3.5-turbo-0613"
user_message = "What is the weather like in Delhi?"

In [ ]:
# A natural language request to the OpenAI API, asking about weather in delhi

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

data = {
   "model": model,
   "messages":[
      {
         "role":"user",
         "content": user_message
      }
   ],
   "functions":[
      {
         "name":"get_current_weather",
         "description":"Get the current weather in a given location",
         "parameters":{
            "type":"object",
            "properties":{
               "location":{
                  "type":"string",
                  "description":"The city and state, e.g. San Francisco, CA"
               },
               "unit":{
                  "type":"string",
                  "enum":[
                     "celsius",
                     "fahrenheit"
                  ]
               }
            },
            "required":[
               "location"
            ]
         }
      }
   ]
}

response = requests.post(open_ai_url, headers=headers, data=json.dumps(data)).json()

JSON(response)

In [ ]:
# Based on the response above we extract the necessary information required to call Weather API
arguments = response['choices'][0]['message']['function_call']['arguments']
city = literal_eval(arguments)['location']

base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={os.getenv('WEATHER_API_KEY')}"

weather_api_response = requests.get(base_url)
weather_api_response = weather_api_response.json()

JSON(weather_api_response)

In [ ]:
# Converting the weather API response back to Natural Language

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

data = {
  "model": model,
  "messages": [
    {"role": "user", "content": user_message},
    {"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": arguments}},
    {"role": "function", "name": "get_current_weather", "content": str(weather_api_response)}
  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}

response = requests.post(open_ai_url, headers=headers, data=json.dumps(data))

print(response.json())

In [ ]:
response.json()['choices'][0]['message']['content']